In [0]:
!pip install uv
!uv add databricks-langchain
!uv sync --active --quiet
dbutils.library.restartPython()

In [0]:
import toml
# from databricks_langchain.uc_ai import (
#     DatabricksFunctionClient,
#     UCFunctionToolkit,
#     set_uc_function_client,
# )
# from databricks.connect import DatabricksSession

In [0]:
env_vars = toml.load("../../conf/env_vars.toml")

In [0]:
from langchain_core.tools import BaseTool, tool

In [0]:
@tool
def get_month_mortality_rate(month: Optional[str | int] = None, year: Optional[str | int] = None) -> str:
    """
    Call to returns the mortality rate for the given month of a year or for the current month.
    """
    if month is None:
        month = datetime.now().strftime("%m")
    if year is None:
        year = datetime.now().strftime("%Y")
    else:
        month = int(month)
        year = int(year)
    feature_store_table_name = f'{env_vars["CATALOG"]}.{env_vars["SCHEMA"]}.srag_features'
    feature_store_table = spark.read.table(feature_store_table_name)
    mortality_rate_percent = feature_store_table.filter(F.year(F.col("DT_EVOLUCA") == year) & F.month(F.col("DT_EVOLUCA") == month)).select(F.col("mortalidade_mensal_perc").last()).collect()

    return f"The mortality rate (%) for {month} of {year} is {mortality_rate_percent}."